In [11]:
import os
import fspbenchmark
import numpy as np
import RSCD
import multiprocessing as mp
import time
current_dir = os.path.dirname(os.path.realpath("example2.ipynb"))
print(current_dir)

/home/du/Workspace/Projects/Scheduling/HFSP


In [12]:
# load instances
ta_ins, ta_solu = fspbenchmark.import_taillard()
ta_brkdlist = fspbenchmark.get_brkd_prob_ta()
arv_times, ptime_list = fspbenchmark.get_new_jobs_ta("low")
print(arv_times[1])

[ 52 522]


## MCTS

In [13]:
# define experiment functions
def exp_fsp1_uct_time_with_seed(param_pipeout, param_dfsp, ptime, time_budget, predict_length, scalar, seed, index, repeat, reward_version, return_version):
    result = RSCD.exp_fsp1_uct_time_with_seed(param_dfsp, ptime, time_budget, predict_length, scalar, seed, index, repeat, reward_version, return_version)
    param_pipeout.send(result)
    # result = [makespan, runtime, number of rescheduling, thread index]

In [14]:
scalar = 5 # coefficient of upper confidence bound
predict_length = 9999999
reward_version = 0 # evaluate makespan of a leaf node without considering dynamic eventss
return_version = 2 # return child node with most visit times
num = 2 # number of threads
for ind in range(0, 1):
    temparray = np.zeros((num, 3))
    print("------", ind, "------")
    ins = ta_ins[ind]
    ext_ins = np.vstack([ins, ptime_list[ind]])
    numj = ins.shape[0]
    nums = ins.shape[1]
    time_budget = numj*nums*0.015
    # hfsp = RSCD.HFSP(nums, numj, ins, np.ones(nums))
    hfsp = RSCD.HFSP(nums, numj, ins, np.ones(nums, dtype=int).tolist())
    coded_solu = ta_solu[ind]
    hfsp.set_solution(coded_solu[1:])
    p_brkd = np.stack([ta_brkdlist[ind][0], ta_brkdlist[ind][1]], axis=0)
    p_effi = np.stack([np.zeros_like(ta_brkdlist[ind][0]), np.zeros_like(ta_brkdlist[ind][1])], axis=0)
    dfsp = RSCD.DFSP1(hfsp, p_brkd, p_effi, arv_times[ind])
    param_list = [dfsp for i in range(num)]
    ins_list = [ext_ins for i in range(num)]
 

    plist = []
    pipeout, pipein = mp.Pipe()

    plist.clear()
    for k in range(num):
        p = mp.Process(target=exp_fsp1_uct_time_with_seed, args=(pipeout, param_list[k], ins_list[k], time_budget, predict_length, scalar, k, k, 10, reward_version, return_version))
        p.start()
        plist.append(p)
    count = 0
    for p in plist:
        foo = pipein.recv()
        print(foo)
        # result_table_ta_seed_uct[ind][int(foo[-1])] = np.asarray(foo)[:-1]
        temparray[int(foo[-1])] = np.asarray(foo)[:-1]
        count += 1
    np.save(current_dir+"/results/ta_newjob/example/uct/ta_"+str(ind+1)+".npy", temparray)


------ 0 ------
[2005.0, 6.709186104, 4.0, 1.0]
[2460.0, 13.430898083, 8.0, 0.0]


## Rules and heuristics

In [15]:
def exp_rs_seed(param_pipeout, param_dfsp, ptime, seed, index):
    result = RSCD.exp_fsp1_rs_with_seed(param_dfsp, ptime, seed, index)
    param_pipeout.send(result)
def exp_spt_seed(param_pipeout, param_dfsp, ptime, seed, index):
    result = RSCD.exp_fsp1_spt_with_seed(param_dfsp, ptime, seed, index)
    param_pipeout.send(result)
def exp_lpt_seed(param_pipeout, param_dfsp, ptime, seed, index):
    result = RSCD.exp_fsp1_lpt_with_seed(param_dfsp, ptime, seed, index)
    param_pipeout.send(result)
def exp_neh_seed(param_pipeout, param_dfsp, ptime, seed, index):
    result = RSCD.exp_fsp1_neh_with_seed(param_dfsp, ptime, seed, index)
    param_pipeout.send(result)
def exp_ritm_seed(param_pipeout, param_dfsp, ptime, seed, index):
    result = RSCD.exp_fsp1_ritm_with_seed(param_dfsp, ptime, seed, index)
    param_pipeout.send(result)

In [16]:
# Take spt for example
num = 2 # number of threads
for ind in range(0, 1):
    temparray = np.zeros((num, 3))
    print("------", ind, "------")
    ins = ta_ins[ind]
    ext_ins = np.vstack([ins, ptime_list[ind]])
    numj = ins.shape[0]
    nums = ins.shape[1]
    hfsp = RSCD.HFSP(nums, numj, ins, np.ones(nums, dtype=int).tolist())
    coded_solu = ta_solu[ind]
    hfsp.set_solution(coded_solu[1:])
    p_brkd = np.stack([ta_brkdlist[ind][0], ta_brkdlist[ind][1]], axis=0)
    p_effi = np.stack([np.zeros_like(ta_brkdlist[ind][0]), np.zeros_like(ta_brkdlist[ind][1])], axis=0)
    dfsp = RSCD.DFSP1(hfsp, p_brkd, p_effi, arv_times[ind])
    param_list = [dfsp for i in range(num)]
    ins_list = [ext_ins for i in range(num)]
 

    plist = []
    pipeout, pipein = mp.Pipe()

    plist.clear()
    for k in range(num):
        p = mp.Process(target=exp_spt_seed, args=(pipeout, param_list[k], ins_list[k],k,k,))
        p.start()
        plist.append(p)

    for p in plist:
        foo = pipein.recv()
        print(foo)
        # result_table_ta_seed_uct[ind][int(foo[-1])] = np.asarray(foo)[:-1]
        temparray[int(foo[-1])] = np.asarray(foo)[:-1]
        count += 1
    np.save(current_dir+"/results/ta_newjob/example/spt/ta_"+str(ind+1)+".npy", temparray)



------ 0 ------
[2465.0, 0.010681591, 8.0, 0.0]
[2217.0, 0.008788696, 4.0, 1.0]


## Metaheuristics

In [17]:
def exp_fsp1_ig_time_with_seed(param_pipeout, param_dfsp, ptime, temperature, d, time_budget, seed, index):
    result = RSCD.exp_fsp1_ig_time_with_seed(param_dfsp, ptime, temperature, d, time_budget, seed, index)
    param_pipeout.send(result)
def exp_fsp1_fta_time_with_seed(param_pipeout, param_dfsp, ptime, temperature, time_budget, seed, index):
    result = RSCD.exp_fsp1_fta_time_with_seed(param_dfsp, ptime, temperature, time_budget, seed, index)
    param_pipeout.send(result)

In [18]:
num = 2 # number of threads
for ind in range(0, 1):
    temparray = np.zeros((num, 3))
    print("------", ind, "------")
    ins = ta_ins[ind]
    temperature = np.average(ins)/10
    ext_ins = np.vstack([ins, ptime_list[ind]])
    numj = ins.shape[0]
    nums = ins.shape[1]
    time_budget = numj*nums*0.015
    # hfsp = RSCD.HFSP(nums, numj, ins, np.ones(nums))
    hfsp = RSCD.HFSP(nums, numj, ins, np.ones(nums, dtype=int).tolist())
    coded_solu = ta_solu[ind]
    hfsp.set_solution(coded_solu[1:])
    p_brkd = np.stack([ta_brkdlist[ind][0], ta_brkdlist[ind][1]], axis=0)
    p_effi = np.stack([np.zeros_like(ta_brkdlist[ind][0]), np.zeros_like(ta_brkdlist[ind][1])], axis=0)
    dfsp = RSCD.DFSP1(hfsp, p_brkd, p_effi, arv_times[ind])
    param_list = [dfsp for i in range(num)]
    ins_list = [ext_ins for i in range(num)]
 

    plist = []
    pipeout, pipein = mp.Pipe()

    plist.clear()
    for k in range(num):
        p = mp.Process(target=exp_fsp1_fta_time_with_seed, args=(pipeout, param_list[k], ins_list[k], temperature, time_budget, k, k))
        p.start()
        plist.append(p)
    count = 0
    for p in plist:
        foo = pipein.recv()
        print(foo)
        # result_table_ta_seed_uct[ind][int(foo[-1])] = np.asarray(foo)[:-1]
        temparray[int(foo[-1])] = np.asarray(foo)[:-1]
        count += 1
    np.save(current_dir+"/results/ta_newjob/example/fta/ta_"+str(ind+1)+".npy", temparray)


------ 0 ------
[2091.0, 6.016784643, 4.0, 1.0]
[2561.0, 12.01126515, 8.0, 0.0]


# Load result

In [19]:
result = np.load(current_dir+"/results/ta_newjob/example/spt/ta_1.npy")
print(result)

[[2.4650000e+03 1.0681591e-02 8.0000000e+00]
 [2.2170000e+03 8.7886960e-03 4.0000000e+00]]
